Import some libraries.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Let's load the csv files.

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')

In [ ]:
train_df.head(10)

In [ ]:
train_df.describe()

Notice that there seems to be some very pricy cars in the million range while most cars seem to be in the ten-thousands ranges. If there are many of these outliers, we need to handle them carefully in order to improve our score.

In [ ]:
train_df[train_df['price'] > 1000000]['price'].hist()

In [ ]:
train_df[train_df['price'] > 1000000].count()

Although less than 1% of the cars in the dataset have a price above a million, these cars will still have a big effect on our score since we are using RMSE which is sensitive to outliers. We will most likely end up underpredicting the price of these cars by a lot since the majority of our cars are in the ten-thousand range.

In [ ]:
test_df.head(10)

After looking at the first few rows of the train and test datasets, the columns 'brand','model_year','milage' and 'fuel_type' seem to be the most straightforward. The 'model' column could potentionally use some feature engineering. For instance, we might want to group 'A6 2.0T Sport' and 'A6 2.0T Premium Plus' as the same model. In any case, it requires more analysis. 

The 'engine' column is somewhat complex and contains information such as horsepower, liter size, engine type and fuel used. We can potentially extract some additional features from this. The same can be said for the 'transmission' column. 

As for 'ext_col' and 'int_col', we might want to group similar colors together. However 'Silver' and 'Silver Ice Metallic' are probably similar colors, but the latter sounds like a potentially more expensive option compared to the former. Finally note that 'int_col' has some entries labelled with '-' so we might have some null columns that we need to take care of. 

The 'accident' and 'clean_title' columns  look like they only have two options each but we need to take care of missing values. 

Let's start by looking into missing values.

In [ ]:
train_df.isna().sum()

In [ ]:
test_df.isna().sum()

We see that missing values are only found in the columns 'fuel_type', 'accident' and 'clean_title'. However we should still check columns to see if any entries are labelled as '-' (or any other variations like 'None','NA', etc.) as in the case of 'int_color' which we saw above.

We might be able to fill in entires for 'fuel_type' using the 'engine' column so let us come back to it later. 

In [ ]:
train_df['accident'].unique()

In [ ]:
test_df['accident'].unique()

Based on the possible values for the 'accident' column, let us fill in the missing values with 'None reported'.

In [ ]:
train_df['accident'] = train_df['accident'].fillna('None reported')
test_df['accident'] = test_df['accident'].fillna('None reported')

In [ ]:
train_df['clean_title'].unique()

In [ ]:
test_df['clean_title'].unique()

Based on the possible values for the 'clean_title' column, let us fill in the missing values with 'No'.

In [ ]:
train_df['clean_title'] = train_df['clean_title'].fillna('No')
test_df['clean_title'] = test_df['clean_title'].fillna('No')

Now let us figure out how to handle 'fuel_type'.

In [ ]:
train_df['fuel_type'].unique()

In [ ]:
test_df['fuel_type'].unique()

The situation is worse than what we initially thought. In addition to missing values, we also have '-' and 'not supported' as entries. A missing value and '-' should be the same thing, but it is unclear what 'not supported' means.

In [ ]:
train_df[train_df['fuel_type']=='not supported']['engine'].unique()

In [ ]:
test_df[test_df['fuel_type']=='not supported']['engine'].unique()

From the different 'engine' entries, it is still unclear what 'not supported' means. Based on the possible values for 'fuel type', 'not supported' could make sense for engines that use Hydrogen Fuel. For the rest of the rows, 'not supported' should probably be replaced with 'Gasoline' since those engines appear to use 'Gasoline'. However we should probably be a little cautious of overly relying on physical intuition due the synthetic nature of the dataset which cause some of these inconsistencies to appear. 

In [ ]:
train_df[train_df['engine'].str.contains('Hydrogen')]

In [ ]:
test_df[test_df['engine'].str.contains('Hydrogen')]

This was not very helpful. We see that Hydrogen fuel engines can take almost every possible value for 'fuel_type'. We also uncovered an additional example of inconsistency in the dataset. The 'Mirai' car model is produced by at least 'Toyota', 'Nissan' and 'Lexus'. This does not occur in the real world. Let's explore the data a bit more before we decide what to do.

In [ ]:
train_df[train_df['engine'].str.contains('Electric Motor')]

In [ ]:
train_df.loc[(train_df['brand'] == 'Tesla') & (train_df['fuel_type'].notna())]

In [ ]:
train_df[train_df['engine'].str.contains('Electric Motor')]['engine'].unique()

In [ ]:
test_df[test_df['engine'].str.contains('Electric Motor')]['engine'].unique()

After examining cars which use some type of electric motor, we should convert the 'fuel_type' for all these cars to 'not supported'. 

In [ ]:
train_df.loc[(train_df['engine'].str.contains('Electric Motor')),'fuel_type'] = 'not supported'

In [ ]:
test_df.loc[(test_df['engine'].str.contains('Electric Motor')),'fuel_type'] = 'not supported'

In [ ]:
train_df[train_df['fuel_type'].isna()]

In [ ]:
test_df[test_df['fuel_type'].isna()]

In [ ]:
train_df[train_df['fuel_type']=='–']

In [ ]:
test_df[test_df['fuel_type']=='–']

After looking at missing values and '–' values, the latter seems easier to deal with. It looks like many of these rows coincide with missing data for the 'engine' column as well. We might be able to fill in this data using the 'brand', 'model', 'model_year' and 'transmission' columns but this could get quite messy.

In [ ]:
train_df.loc[(train_df['fuel_type'] == '–') & (train_df['engine'] == '–')]

It will mostly likely require more research to deal with these entries. As a first pass, let us just ignore this issue and just fill in missing values with '–' as well. 

In [ ]:
train_df['fuel_type'] = train_df['fuel_type'].fillna('–')
test_df['fuel_type'] = test_df['fuel_type'].fillna('–')

Let us now deal with the 'engine' column. We will extract three new features from this column: 'horsepower', 'engine_size', 'cylinder_num'. Unfortunately not every 'engine' entry contains all this information so we will have to figure out how to fill in this missing data later.

In [ ]:
def preprocess_engine(df):
    df['horsepower'] = df['engine'].str.extract('(\d+\.?\d*)HP').astype(float)
    df['engine_size'] = df['engine'].str.extract('(\d+\.?\d*)L').astype(float)
    df['cylinder_num'] = df['engine'].str.extract('(\d+)\s*Cylinder').astype(float)
    
    return df

In [ ]:
preprocess_engine(train_df)
preprocess_engine(test_df)

In [ ]:
train_df['horsepower'].hist()

In [ ]:
test_df['horsepower'].hist()

Looking at the distribution for horsepower, let us fill in missing values with the median. Presumably the higher values we see are due to sports cars which skew the data. 

In [ ]:
train_df['horsepower'] = train_df['horsepower'].fillna(round(train_df['horsepower'].median(), 1))
test_df['horsepower'] = test_df['horsepower'].fillna(round(test_df['horsepower'].median(), 1))

In [ ]:
train_df['engine_size'].hist()

In [ ]:
test_df['engine_size'].hist()

There is one extremely common value so let us fill in the data with the most common value.

In [ ]:
train_df['engine_size'] = train_df['engine_size'].fillna(train_df['engine_size'].value_counts().index[0])
test_df['engine_size'] = test_df['engine_size'].fillna(test_df['engine_size'].value_counts().index[0])


In [ ]:
train_df['cylinder_num'].hist()

In [ ]:
test_df['cylinder_num'].hist()

Let us also take the most common value for 'cylinder_size'.

In [ ]:
train_df['cylinder_num'] = train_df['cylinder_num'].fillna(train_df['cylinder_num'].value_counts().index[0])
test_df['cylinder_num'] = test_df['cylinder_num'].fillna(test_df['cylinder_num'].value_counts().index[0])

Now let's handle the 'transmission' column. 

In [ ]:
train_df['transmission'].unique()

In [ ]:
test_df['transmission'].unique()

We see that the are a lot of redundant entries in this column. For instance, '8-Speed A/T', '8-SPEED AT', '8-SPEED A/T' are all the same. We can try to consolidate these columns. Furthermore 'AT' and its variations stand for 'Automatic' while 'MT' stands for 'Manual'. Also note that there is a 'F'-type transmission which does not exist in the real world. 

In [ ]:
def preprocess_transmission(df):
    df.transmission = df.transmission.replace('A/T','Automatic',regex = True)
    df.transmission = df.transmission.replace('M/T','Manual',regex = True)
    
    df.transmission = df.transmission.replace('CVT Transmission', 'Automatic CVT', regex=True)
    df.transmission = df.transmission.replace('CVT-F', 'F', regex=True)
    df.transmission = df.transmission.replace('Variable', 'Automatic CVT', regex=True )

    df.transmission = df.transmission.replace('Transmission Overdrive Switch', 'Automatic', regex=True)

    df.transmission = df.transmission.replace('8-SPEED AT', '8-Speed Automatic', regex=True)
    df.transmission = df.transmission.replace('8-SPEED Automatic', '8-Speed Automatic', regex=True)

    df.transmission = df.transmission.replace('6 Speed At/Mt', '6-Speed Automatic with Auto-Shift', regex=True)
    df.transmission = df.transmission.replace('6-Speed Electronically Controlled Automatic with O', '6-Speed Automatic with Auto-Shift', regex=True)

    df.transmission = df.transmission.replace('6 Speed Mt', '6-Speed Manual', regex=True)
    
    return df

In [ ]:
preprocess_transmission(train_df)
preprocess_transmission(test_df)

Let's take a step back and think more about our target variable 'price'.

In [ ]:
sns.histplot(data = np.log1p(train_df['price']))

In [ ]:
sns.boxplot(x = train_df['price'])

In [ ]:
(train_df[train_df['price'] > 1000000].groupby('brand')['id'].count()).divide(train_df.groupby('brand')['id'].count()).fillna(0).nlargest(20)

In [ ]:
plt.figure(figsize = (20,10))
brand_counts = train_df['brand'].value_counts()
sns.barplot(y=brand_counts.index, x=brand_counts.values)

In [ ]:
train_df.groupby('brand')['price'].mean().sort_values(ascending = False).iloc[:20]

In [ ]:
(train_df[train_df['price'] > 100000].groupby('model')['id'].count()).divide(train_df.groupby('model')['id'].count()).fillna(0).nlargest(20)

In [ ]:
train_df.groupby('model')['price'].mean().sort_values()

In [ ]:
sns.histplot(data = train_df.groupby('model')['price'].mean(),bins= 30)

In [ ]:
(train_df[train_df['price'] > 100000].groupby('ext_col')['id'].count()).divide(train_df.groupby('ext_col')['id'].count()).fillna(0).nlargest(20)

In [ ]:
train_df.groupby('ext_col')['price'].mean().sort_values(ascending = False).iloc[:20]

In [ ]:
sns.histplot(data = train_df.groupby('ext_col')['price'].mean(),bins = 30)

In [ ]:
(train_df[train_df['price'] > 100000].groupby('int_col')['id'].count()).divide(train_df.groupby('int_col')['id'].count()).fillna(0).nlargest(20)

In [ ]:
train_df.groupby('int_col')['price'].mean().sort_values(ascending = False).iloc[:20]

In [ ]:
sns.histplot(data = train_df.groupby('int_col')['price'].mean(),bins = 30)

We will take some of the categorical features that have a large number of values and try to highlight some of the values which have a high proportion in more expensive cars which cost more than a million. This is not necessarily the best method since high proportion does not necessarily imply causation and we can have some data balance issues where some of the values we single out have relatively low occurences. 

In [ ]:
exp_brand = ['Bugatti','Lamborghini','Rolls-Royce','Bentley','McLaren','Ferrari','Aston']

#exp_model = ['Carrera GT Base','Rover LR4 Lux','Cayman GTS','Martin DB7 Vantage Volante',
 #            'Continental GT GT Speed','GT','911 R','Aventador LP750-4 Superveloce',
  #           'Gallardo LP570-4 Superleggera','Gallardo SE','911 Turbo Cabriolet']

#exp_ext_col = ['Blue Caelum','Dark Sapphire','Bianco Monocerus','C / C',
 #              'Shadow Black','Sandstone Metallic','China Blue','Ice',
  #             'Donington Grey Metallic','Onyx','Nero Noctis']
#exp_int_col = ['Dark Auburn','Hotspur','Shale','Camel Leather','Linen',
 #              'Black / Brown','Brandy','Beluga Hide','Adrenaline Red',
  #             'Portland','Yellow']


#def preprocess_expensive(df):
 #   df['exp_brand'] = df['brand'].apply(lambda x: 1 if x in exp_brand else 0)
  #  df['exp_model'] = df['model'].apply(lambda x: 1 if x in exp_model else 0)
   # df['exp_ext_color'] = df['ext_col'].apply(lambda x: 1 if x in exp_ext_col else 0)
    #df['exp_int_color'] = df['int_col'].apply(lambda x: 1 if x in exp_int_col else 0)

grouped_models = train_df.groupby('model')['price'].mean()
grouped_ext_col = train_df.groupby('ext_col')['price'].mean()
grouped_int_col = train_df.groupby('int_col')['price'].mean()

def preprocess_expensive(df):
    df['exp_brand'] = df['brand'].apply(lambda x: 1 if x in exp_brand else 0)
    df['exp_model'] = df['model'].apply(lambda x: 1 if x in grouped_models and grouped_models[x] > 100000 else 0)
    df['exp_ext_col'] = df['ext_col'].apply(lambda x: 1 if x in grouped_ext_col and grouped_ext_col[x] > 100000 else 0)
    df['exp_int_col'] = df['int_col'].apply(lambda x: 1 if x in grouped_int_col and grouped_int_col[x] > 100000 else 0)
    

In [ ]:
preprocess_expensive(train_df)
preprocess_expensive(test_df)

In [ ]:
train_df['ext_col'].nunique()

In [ ]:
test_df['ext_col'].nunique()

There are a ton of colors. We could keep the most common colors and bundle the rest of the colors as 'other' but that probably loses a lot of information.

In [ ]:
X = train_df.drop(['id', 'engine'], axis=1).copy()
y = X.pop('price')
X_test = test_df.drop(['id', 'engine'], axis=1).copy()

In [ ]:

from sklearn.preprocessing import LabelEncoder

def encoding(df):
    cat_columns = df.select_dtypes(include='object').columns
    
    encoder = LabelEncoder()
    for column in cat_columns:
        df[column] = encoder.fit_transform(df[column])
    
    return df


In [ ]:
X = encoding(X)
X_test = encoding(X_test)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.20,
    random_state=17
)

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import optuna

def objective(trial):
    xgb_params = dict(
        objective='reg:squarederror', 
        eval_metric='rmse', 
        max_depth=trial.suggest_int("max_depth", 2, 10),
        learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        n_estimators=trial.suggest_int("n_estimators", 1000, 8000),
        min_child_weight=trial.suggest_int("min_child_weight", 1, 10),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.2, 1.0),
        subsample=trial.suggest_float("subsample", 0.2, 1.0),
        reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e2, log=True),
        reg_lambda=trial.suggest_float("reg_lambda", 1e-4, 1e2, log=True),
    )
    
    model = XGBRegressor(**xgb_params)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
xgb_params = study.best_params

In [ ]:
best_model = XGBRegressor(**xgb_params)
best_model.fit(X_train, y_train)

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
submission = pd.read_csv("/kaggle/input/playground-series-s4e9/sample_submission.csv")

In [ ]:
submission['price'] = y_pred

In [ ]:
submission.to_csv("submission.csv", index=False)